<a href="https://colab.research.google.com/github/mhoangvslev/bricklink_tool_suites/blob/main/Bricklink_Compare_Parts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

deagle_v1 = pd.read_xml("Desert Eagle v1.xml")
deagle_v2 = pd.read_xml("Desert Eagle v2.xml")

# A litte bit of cleaning

In [ ]:
deagle_v1['ITEMID'] = deagle_v1['ITEMID'].astype(str)
deagle_v1

In [ ]:
deagle_v2['ITEMID'] = deagle_v2['ITEMID'].astype(str)
deagle_v2

In [ ]:
merged_df = deagle_v1.merge(deagle_v2, on=["ITEMID", "COLOR"], how="right", suffixes=["_v1", "_v2"])#.dropna()
merged_df

In [ ]:
#@title Get all the parts that are not in v1
new_parts = merged_df[merged_df['ITEMTYPE_v1'].isnull()]


In [ ]:
#@title Get the difference in quantity for parts that are present in both but with greater quantity in v2

# Filter the rows where the MINQTY in list B is greater than MINQTY in list A
extra_parts = merged_df[merged_df['MINQTY_v2'] > merged_df['MINQTY_v1']]

# One might only want to order the extra parts only
extra_parts["MINQTY"] = (extra_parts["MINQTY_v2"] - extra_parts["MINQTY_v1"]).astype(int)
extra_parts

In [ ]:
new_order = pd.concat([new_parts, extra_parts], ignore_index=True)
new_order['MINQTY'] = new_order['MINQTY'].fillna(new_order['MINQTY_v2']).astype(int)
# Drop columns with suffix "_v1"
new_order = new_order.filter(regex=r'^(?!.*_v1$)')

# Remove suffix "_v2" from column names
new_order.drop('MINQTY_v2', axis=1, inplace=True)
new_order.columns = new_order.columns.str.replace(r'_v2$', '', regex=True)
new_order

In [ ]:
xml_output = new_order.to_xml(root_name='INVENTORY', row_name="ITEM", prefix=None, xml_declaration=True, index=False)
# Write the XML data to a file
with open('new_order.xml', 'w') as file:
    file.write(xml_output)